In [2]:
import numpy as np
import matplotlib.pyplot as plt
import skimage as ski
from skimage.feature import blob_log
from skimage.filters import gaussian
from skimage.data import cells3d
from skimage import filters
import glob
import stackview
import scipy as sp
from napari_segment_blobs_and_things_with_membranes import split_touching_objects
import pyclesperanto as cle
import pandas as pd


c:\Users\grufl\Desktop\211007 Sicherung USB\uni\Prakt bioinfo\Bioinfo\.venv\Lib\site-packages\napari_tools_menu\__init__.py:10: UserWarning: Importing QT failed; now introducing dummy definitions of QMenu class and register_function decorator.
  warnings.warn("Importing QT failed; now introducing dummy definitions of QMenu class and register_function decorator.")


In [16]:
#Atto_425 = A
#Alexa_488 = G
#Alexa_568 = C
#Alexa_647 = T
first_file_select = sorted(glob.glob(r"C:\Users\grufl\Desktop\211007 Sicherung USB\uni\Prakt bioinfo\Bioinfo\data\raw\selected-tiles\out_opt_flow_registered_X10_Y10_*.tif"))
file_c01 = first_file_select[:4]
file_c02 = first_file_select[6:10]
print(file_c02)

['C:\\Users\\grufl\\Desktop\\211007 Sicherung USB\\uni\\Prakt bioinfo\\Bioinfo\\data\\raw\\selected-tiles\\out_opt_flow_registered_X10_Y10_c02_Alexa_488.tif', 'C:\\Users\\grufl\\Desktop\\211007 Sicherung USB\\uni\\Prakt bioinfo\\Bioinfo\\data\\raw\\selected-tiles\\out_opt_flow_registered_X10_Y10_c02_Alexa_568.tif', 'C:\\Users\\grufl\\Desktop\\211007 Sicherung USB\\uni\\Prakt bioinfo\\Bioinfo\\data\\raw\\selected-tiles\\out_opt_flow_registered_X10_Y10_c02_Alexa_647.tif', 'C:\\Users\\grufl\\Desktop\\211007 Sicherung USB\\uni\\Prakt bioinfo\\Bioinfo\\data\\raw\\selected-tiles\\out_opt_flow_registered_X10_Y10_c02_Atto_425.tif']


In [6]:
first_file_select = sorted(glob.glob(r"C:\Users\grufl\Desktop\211007 Sicherung USB\uni\Prakt bioinfo\Bioinfo\data\raw\selected-tiles\out_opt_flow_registered_X10_Y10_c01_*.tif"))
file_select = first_file_select[:4]
print(file_select)
images = [ski.io.imread(file) for file in file_select]
image_data = np.stack(images, axis=0)
# Maximum Intensity Projection
mip = np.max(image_data, axis=0)
# labeling von MIP
image_denoised = gaussian(mip, sigma=2, preserve_range=True)
image_binary = image_denoised > ski.filters.threshold_li(image_denoised)
split_objects = split_touching_objects(image_binary)
labeled_objects = ski.measure.label(split_objects, connectivity=1)
mip_filtered_labels = cle.exclude_labels_outside_size_range(labeled_objects, minimum_size=150, maximum_size=2000)
mip_num_features = mip_filtered_labels.max()
print(mip_num_features)

['C:\\Users\\grufl\\Desktop\\211007 Sicherung USB\\uni\\Prakt bioinfo\\Bioinfo\\data\\raw\\selected-tiles\\out_opt_flow_registered_X10_Y10_c01_Alexa_488.tif', 'C:\\Users\\grufl\\Desktop\\211007 Sicherung USB\\uni\\Prakt bioinfo\\Bioinfo\\data\\raw\\selected-tiles\\out_opt_flow_registered_X10_Y10_c01_Alexa_568.tif', 'C:\\Users\\grufl\\Desktop\\211007 Sicherung USB\\uni\\Prakt bioinfo\\Bioinfo\\data\\raw\\selected-tiles\\out_opt_flow_registered_X10_Y10_c01_Alexa_647.tif', 'C:\\Users\\grufl\\Desktop\\211007 Sicherung USB\\uni\\Prakt bioinfo\\Bioinfo\\data\\raw\\selected-tiles\\out_opt_flow_registered_X10_Y10_c01_Atto_425.tif']
159.0


In [7]:
#15.2

# get region properties
props = ski.measure.regionprops(mip_filtered_labels)

# get bounding boxes for each label
bboxes = [prop.bbox for prop in props]
print("Bounding boxes:", bboxes)

Bounding boxes: [(4, 321, 24, 347), (7, 633, 26, 666), (8, 901, 28, 920), (10, 876, 32, 891), (14, 193, 40, 231), (14, 959, 32, 981), (21, 333, 36, 350), (24, 938, 37, 959), (26, 622, 69, 665), (30, 949, 59, 982), (31, 859, 47, 885), (39, 134, 58, 155), (42, 860, 68, 901), (52, 425, 71, 439), (55, 144, 71, 159), (57, 349, 83, 382), (66, 123, 88, 151), (69, 405, 96, 424), (71, 456, 104, 494), (75, 187, 101, 216), (92, 783, 122, 815), (99, 739, 120, 753), (113, 121, 144, 138), (114, 344, 145, 369), (118, 798, 142, 821), (120, 101, 150, 119), (120, 230, 154, 247), (121, 250, 142, 267), (138, 511, 162, 533), (175, 42, 199, 52), (177, 489, 195, 505), (184, 0, 223, 14), (186, 238, 204, 252), (193, 275, 218, 292), (210, 19, 225, 36), (222, 538, 248, 575), (223, 573, 269, 592), (230, 593, 252, 614), (238, 263, 291, 307), (243, 488, 297, 515), (253, 517, 266, 533), (258, 701, 288, 718), (282, 512, 322, 528), (282, 720, 297, 736), (287, 694, 303, 707), (292, 535, 318, 550), (300, 87, 323, 122), 

In [8]:
channel_analysis = {}
for box in bboxes:
    best_fit = None
    best_fit_index = None
    min_row, min_col, max_row, max_col = box
    for idx, image in enumerate(images):
        roi = image[min_row:max_row, min_col:max_col]
        roi_mean = roi.mean()
        if best_fit is None:
            best_fit = roi
            best_fit_index = idx
        elif roi_mean > best_fit.mean():
            best_fit = roi
            best_fit_index = idx
    channel_analysis[box] = best_fit_index
print(channel_analysis)

{(4, 321, 24, 347): 2, (7, 633, 26, 666): 2, (8, 901, 28, 920): 1, (10, 876, 32, 891): 2, (14, 193, 40, 231): 2, (14, 959, 32, 981): 2, (21, 333, 36, 350): 2, (24, 938, 37, 959): 2, (26, 622, 69, 665): 2, (30, 949, 59, 982): 2, (31, 859, 47, 885): 2, (39, 134, 58, 155): 2, (42, 860, 68, 901): 2, (52, 425, 71, 439): 1, (55, 144, 71, 159): 2, (57, 349, 83, 382): 1, (66, 123, 88, 151): 2, (69, 405, 96, 424): 2, (71, 456, 104, 494): 2, (75, 187, 101, 216): 1, (92, 783, 122, 815): 2, (99, 739, 120, 753): 1, (113, 121, 144, 138): 2, (114, 344, 145, 369): 2, (118, 798, 142, 821): 2, (120, 101, 150, 119): 2, (120, 230, 154, 247): 2, (121, 250, 142, 267): 1, (138, 511, 162, 533): 2, (175, 42, 199, 52): 2, (177, 489, 195, 505): 1, (184, 0, 223, 14): 1, (186, 238, 204, 252): 2, (193, 275, 218, 292): 2, (210, 19, 225, 36): 2, (222, 538, 248, 575): 2, (223, 573, 269, 592): 1, (230, 593, 252, 614): 2, (238, 263, 291, 307): 2, (243, 488, 297, 515): 1, (253, 517, 266, 533): 2, (258, 701, 288, 718): 2,

In [9]:
A_count = 0
G_count = 0
C_count = 0
T_count = 0
for box in channel_analysis:
    if channel_analysis[box] == 0:
        G_count += 1
    elif channel_analysis[box] == 1:
        C_count += 1
    elif channel_analysis[box] == 2:
        T_count += 1
    elif channel_analysis[box] == 3:
        A_count += 1
print("A:", A_count)
print("G:", G_count)
print("C:", C_count)
print("T:", T_count)

A: 0
G: 0
C: 40
T: 119


In [ ]:
first_file_select = sorted(glob.glob(r"C:\Users\grufl\Desktop\211007 Sicherung USB\uni\Prakt bioinfo\Bioinfo\data\raw\selected-tiles\out_opt_flow_registered_X10_Y10_*.tif"))
CounterA = 0
CounterB = 4
barcode = []

while CounterB <= 22:
    file_select = first_file_select[CounterA:CounterB]
    images = [ski.io.imread(file) for file in file_select]
    image_data = np.stack(images, axis=0)
    # Maximum Intensity Projection
    mip = np.max(image_data, axis=0)
    # labeling von MIP
    image_denoised = gaussian(mip, sigma=2, preserve_range=True)
    image_binary = image_denoised > ski.filters.threshold_li(image_denoised)
    split_objects = split_touching_objects(image_binary)
    labeled_objects = ski.measure.label(split_objects, connectivity=1)
    mip_filtered_labels = cle.exclude_labels_outside_size_range(labeled_objects, minimum_size=150, maximum_size=2000)
    mip_num_features = mip_filtered_labels.max()
    print("Number of features in MIP:", mip_num_features)
    # get region properties
    props = ski.measure.regionprops(mip_filtered_labels)
    print("Number of properties:", len(props))
    # get bounding boxes for each label
    bboxes = [prop.bbox for prop in props]
    print("Bounding boxes:", bboxes)

    channel_analysis = {}
    for box in bboxes:
        best_fit = None
        best_fit_index = None
        min_row, min_col, max_row, max_col = box
        for idx, image in enumerate(images):
            roi = image[min_row:max_row, min_col:max_col]
            roi_mean = roi.mean()
            if best_fit is None:
                best_fit = roi
                best_fit_index = idx
            elif roi_mean > best_fit.mean():
                best_fit = roi
                best_fit_index = idx
        channel_analysis[box] = best_fit_index
    print(channel_analysis)

    A_count = 0
    G_count = 0
    C_count = 0
    T_count = 0
    for box in channel_analysis:
        if channel_analysis[box] == 0:
            G_count += 1
        elif channel_analysis[box] == 1:
            C_count += 1
        elif channel_analysis[box] == 2:
            T_count += 1
        elif channel_analysis[box] == 3:
            A_count += 1
    if A_count > G_count and A_count > C_count and A_count > T_count:
        barcode.append("A")
    elif G_count > A_count and G_count > C_count and G_count > T_count:
        barcode.append("G")
    elif C_count > A_count and C_count > G_count and C_count > T_count:
        barcode.append("C")
    elif T_count > A_count and T_count > G_count and T_count > C_count:
        barcode.append("T")
    print("A:", A_count)
    print("G:", G_count)
    print("C:", C_count)
    print("T:", T_count)
    CounterA += 6
    CounterB += 6
print("Barcode:", barcode)

Number of features in MIP: 159.0
Number of properties: 159
Bounding boxes: [(4, 321, 24, 347), (7, 633, 26, 666), (8, 901, 28, 920), (10, 876, 32, 891), (14, 193, 40, 231), (14, 959, 32, 981), (21, 333, 36, 350), (24, 938, 37, 959), (26, 622, 69, 665), (30, 949, 59, 982), (31, 859, 47, 885), (39, 134, 58, 155), (42, 860, 68, 901), (52, 425, 71, 439), (55, 144, 71, 159), (57, 349, 83, 382), (66, 123, 88, 151), (69, 405, 96, 424), (71, 456, 104, 494), (75, 187, 101, 216), (92, 783, 122, 815), (99, 739, 120, 753), (113, 121, 144, 138), (114, 344, 145, 369), (118, 798, 142, 821), (120, 101, 150, 119), (120, 230, 154, 247), (121, 250, 142, 267), (138, 511, 162, 533), (175, 42, 199, 52), (177, 489, 195, 505), (184, 0, 223, 14), (186, 238, 204, 252), (193, 275, 218, 292), (210, 19, 225, 36), (222, 538, 248, 575), (223, 573, 269, 592), (230, 593, 252, 614), (238, 263, 291, 307), (243, 488, 297, 515), (253, 517, 266, 533), (258, 701, 288, 718), (282, 512, 322, 528), (282, 720, 297, 736), (287, 

In [24]:
first_file_select = sorted(glob.glob(r"C:\Users\grufl\Desktop\211007 Sicherung USB\uni\Prakt bioinfo\Bioinfo\data\raw\selected-tiles\out_opt_flow_registered_X10_Y10_*.tif"))
CounterA = 0
CounterB = 4
barcode = []

while CounterB <= 22:
    file_select = first_file_select[CounterA:CounterB]
    images = [ski.io.imread(file) for file in file_select]
    image_data = np.stack(images, axis=0)
    # Maximum Intensity Projection
    mip = np.max(image_data, axis=0)
    # labeling von MIP
    image_denoised = gaussian(mip, sigma=2, preserve_range=True)
    image_binary = image_denoised > ski.filters.threshold_li(image_denoised)
    split_objects = split_touching_objects(image_binary)
    labeled_objects = ski.measure.label(split_objects, connectivity=1)
    mip_filtered_labels = cle.exclude_labels_outside_size_range(labeled_objects, minimum_size=150, maximum_size=2000)
    mip_num_features = mip_filtered_labels.max()
    print("Number of features in MIP:", mip_num_features)
    # get region properties
    props = ski.measure.regionprops(mip_filtered_labels)
    print("Number of properties:", len(props))
    # get bounding boxes for each label
    bboxes = [prop.bbox for prop in props]
    print("Bounding boxes:", bboxes)

    channel_analysis = {}
    for box in bboxes:
        best_fit = None
        best_fit_index = None
        min_row, min_col, max_row, max_col = box
        for idx, image in enumerate(images):
            roi = image[min_row:max_row, min_col:max_col]
            roi_mean = roi.mean()
            if roi_mean < (mip[min_row:max_row, min_col:max_col].mean()*0.98):
                pass
            elif best_fit is None:
                best_fit = roi
                best_fit_index = idx
            elif roi_mean > best_fit.mean():
                best_fit = roi
                best_fit_index = idx
        channel_analysis[box] = best_fit_index
    print(channel_analysis)

    A_count = 0
    G_count = 0
    C_count = 0
    T_count = 0
    for box in channel_analysis:
        if channel_analysis[box] == 0:
            G_count += 1
        elif channel_analysis[box] == 1:
            C_count += 1
        elif channel_analysis[box] == 2:
            T_count += 1
        elif channel_analysis[box] == 3:
            A_count += 1
    if A_count > G_count and A_count > C_count and A_count > T_count:
        barcode.append("A")
    elif G_count > A_count and G_count > C_count and G_count > T_count:
        barcode.append("G")
    elif C_count > A_count and C_count > G_count and C_count > T_count:
        barcode.append("C")
    elif T_count > A_count and T_count > G_count and T_count > C_count:
        barcode.append("T")
    print("A:", A_count)
    print("G:", G_count)
    print("C:", C_count)
    print("T:", T_count)
    CounterA += 6
    CounterB += 6
print("Barcode:", barcode)

Number of features in MIP: 159.0
Number of properties: 159
Bounding boxes: [(4, 321, 24, 347), (7, 633, 26, 666), (8, 901, 28, 920), (10, 876, 32, 891), (14, 193, 40, 231), (14, 959, 32, 981), (21, 333, 36, 350), (24, 938, 37, 959), (26, 622, 69, 665), (30, 949, 59, 982), (31, 859, 47, 885), (39, 134, 58, 155), (42, 860, 68, 901), (52, 425, 71, 439), (55, 144, 71, 159), (57, 349, 83, 382), (66, 123, 88, 151), (69, 405, 96, 424), (71, 456, 104, 494), (75, 187, 101, 216), (92, 783, 122, 815), (99, 739, 120, 753), (113, 121, 144, 138), (114, 344, 145, 369), (118, 798, 142, 821), (120, 101, 150, 119), (120, 230, 154, 247), (121, 250, 142, 267), (138, 511, 162, 533), (175, 42, 199, 52), (177, 489, 195, 505), (184, 0, 223, 14), (186, 238, 204, 252), (193, 275, 218, 292), (210, 19, 225, 36), (222, 538, 248, 575), (223, 573, 269, 592), (230, 593, 252, 614), (238, 263, 291, 307), (243, 488, 297, 515), (253, 517, 266, 533), (258, 701, 288, 718), (282, 512, 322, 528), (282, 720, 297, 736), (287, 